In [16]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt
import ssm
from sklearn import preprocessing
from sklearn.model_selection import KFold
from scipy import stats

from notebooks.imports import *
from config import dir_config, main_config
from src.utils.glm_hmm_utils import *
import pickle
import copy

### Configuration

In [18]:
from config import dir_config, main_config

raw_dir = Path(dir_config.data.raw)
processed_dir = Path(dir_config.data.processed)

metadata = pd.read_csv(Path(processed_dir, "processed_metadata_accu_60.csv"))
data = pd.read_csv(Path(processed_dir, "processed_data_accu_60_all.csv"))

In [19]:
experiment_sites = ["Stanford"]

metadata = metadata[metadata['experiment_site'].isin(experiment_sites)].reset_index(drop=True)
data = data[data['subject_id'].isin(metadata['subject_id'])].reset_index(drop=True)

# add session_id to data with matching subject_id and medication
metadata['session_id'] = metadata[['subject_id', 'treatment']].apply(lambda x: '_'.join(x.astype(str).str.upper()), axis=1)
data['session_id'] = data[['subject_id', 'medication']].apply(lambda x: '_'.join(x.astype(str).str.upper()), axis=1)
data.choice = data.choice.fillna(-1).astype(int)
data.target = data.target.fillna(-1).astype(int)
data.outcome = data.outcome.fillna(-1).astype(int)

### Helper Functions

In [20]:
def extract_previous_trial_data(trial_data, invalid_idx):
    npr.seed(1)
    prev_choice = np.hstack([trial_data.choice[0], trial_data.choice[:-1]])  # 0:awayPrior, 1:toPrior of previous valid trial
    prev_target = np.hstack([trial_data.target[0], trial_data.target[:-1]]) * 2 - 1  # 0:awayPrior, 1:toPrior of previous valid trial
    prev_color = np.hstack([trial_data.color[0], trial_data.color[:-1]])  # 0:equalPrior, 1:UnequalPrior of previous valid trial

    # indices where the previous trial is invalid/valid
    prev_invalid_idx = np.array(invalid_idx) + 1
    if 0 in invalid_idx:
        prev_invalid_idx = np.append(0, prev_invalid_idx)
    prev_valid_idx = np.setdiff1d(np.arange(len(trial_data)), prev_invalid_idx)

    for i in prev_invalid_idx[prev_invalid_idx < len(trial_data)]:
        if i < prev_valid_idx[0]: #randomly sample if no previous valid trials
            prev_choice[i] = np.random.binomial(1,0.5)
            prev_target[i] = np.random.binomial(1,0.5) * 2 - 1
            prev_color[i]  = np.random.binomial(1,0.5)
        else:
            last_valid =  np.where(prev_valid_idx<i)[0][-1]
            prev_choice[i] = prev_choice[prev_valid_idx[last_valid]]
            prev_target[i] = prev_target[prev_valid_idx[last_valid]]
            prev_color[i] = prev_color[prev_valid_idx[last_valid]]

    prev_choice = (prev_choice * 2) - 1 # -1:awayPrior, 1:toPrior of previous valid trial
    return prev_choice.astype(int), prev_target.astype(int), prev_color.astype(int)

def prepare_input_data(data, input_dim, invalid_idx):
    X = np.ones((1, data.shape[0], input_dim))

    X[0,:,0] = data.signed_coherence / 100
    X[0,:,1] = data.color

    prev_choice, prev_target, prev_color = extract_previous_trial_data(data, invalid_idx)
    if input_dim == 4:
        X[0,:,3] = prev_choice
    elif input_dim == 5:
        X[0,:,3] = prev_choice
        X[0,:,4] = prev_target
    elif input_dim == 6:
        X[0,:,3] = prev_choice
        X[0,:,4] = prev_target
        X[0,:,5] = prev_color
    return list(X)

### Data processing

In [21]:
print("------------- info ----------------")
print(data.info())
print("------------- Head ----------------")
print(data.head())
print("\n\n------------- describe ----------------\n\n")
print(data.describe())
print("------------- nan counts ----------------")
print(data.isnull().sum())
print("\n\n------------- dtypes ----------------\n\n")
print(data.dtypes)
print("\n\n------------- shape ----------------\n\n")
print(data.shape)

------------- info ----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20585 entries, 0 to 20584
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   subject_id        20585 non-null  object 
 1   medication        20585 non-null  object 
 2   prior             20585 non-null  object 
 3   prior_direction   20585 non-null  object 
 4   prior_color       20585 non-null  object 
 5   color             20307 non-null  float64
 6   coherence         20307 non-null  float64
 7   target            20585 non-null  int64  
 8   is_valid          20585 non-null  bool   
 9   outcome           20585 non-null  int64  
 10  choice            20585 non-null  int64  
 11  reaction_time     20585 non-null  float64
 12  session_filename  20585 non-null  object 
 13  signed_coherence  20307 non-null  float64
 14  session_id        20585 non-null  object 
dtypes: bool(1), float64(4), int64(3), object(7)
memory 

#### Data preparation

#### Create design matrix (input, output, mask)

In [22]:
n_states = 2       # number of discrete states
obs_dim = 1           # number of observed dimensions: choice(toPrior/awayPrior)
num_categories = 2    # number of categories for output
input_dim = 5        # input dimensions: current signed coherence, current stimulus color, 1(bias), previous choice(toPrior/awayPrior), previous target side(toPrior/awayPrior), previous color(toPrior/awayPrior)

In [ ]:
inputs_session_wise = []
choices_session_wise = []
invalid_idx_session_wise = []
masks_session_wise = []
reaction_time_session_wise = []

# off medication sessions
for session in metadata.session_id.unique():
    session_data = data[data['session_id'] == session].reset_index(drop=True)


    invalid_idx = np.where(session_data.outcome < 0)[0]
    valid_idx = np.where(session_data.outcome >= 0)[0]

    inputs = prepare_input_data(session_data, input_dim, invalid_idx)
    choices = session_data.choice.values.reshape(-1,1).astype('int')

    # for training, replace -1 with random sample from 0,1
    choices[choices == -1] = npr.choice([0,1],invalid_idx.shape[0])
    mask = np.ones_like(choices, dtype=bool)
    mask[invalid_idx] = 0
    reaction_time = np.array(session_data.reaction_time)

    masks_session_wise.append(mask)
    inputs_session_wise += inputs
    choices_session_wise.append(choices)
    reaction_time_session_wise.append(reaction_time)

inputs_aggregate, choices_aggregate, masks_aggregate = [], [], []
inputs_aggregate.append(np.vstack(inputs_session_wise))
choices_aggregate.append(np.vstack(choices_session_wise))
masks_aggregate.append(np.vstack(masks_session_wise))

unnormalized_inputs = copy.deepcopy(inputs_session_wise)

# scaling signed coherence
inputs_aggregate[0][masks_aggregate[0][:,0],0] = preprocessing.scale(inputs_aggregate[0][masks_aggregate[0][:,0],0], axis=0)
for idx in range(len(metadata.session_id.unique())):
    inputs_session_wise[idx][masks_session_wise[idx][:,0],0] = preprocessing.scale(inputs_session_wise[idx][masks_session_wise[idx][:,0],0], axis=0)


In [24]:
models_glm_hmm, fit_lls_glm_hmm = global_fit(choices_aggregate, inputs_aggregate, masks=masks_aggregate, n_iters= 1000, n_initializations=20)

Fitting GLM globally...


  0%|          | 0/1000 [00:00<?, ?it/s]

Fitting 2 states...


LP: -10651.3:   0%|          | 0/1000 [00:00<?, ?it/s]

Fitting 3 states...


LP: -10595.9:   0%|          | 1/1000 [00:00<02:00,  8.28it/s]

Fitting 4 states...


LP: -10699.4:   0%|          | 0/1000 [00:00<?, ?it/s]

Fitting 5 states...


LP: -9845.6: 100%|██████████| 1000/1000 [02:09<00:00,  7.69it/s]


In [25]:
# get best model of 20 initializations for each state
init_params = {
    'glm_weights': {},
    'transition_matrices': {}
}
for n_states in np.arange(2,6):
    best_idx = fit_lls_glm_hmm[n_states].index(max(fit_lls_glm_hmm[n_states]))
    init_params['glm_weights'][n_states] = models_glm_hmm[n_states][best_idx].observations.params
    init_params['transition_matrices'][n_states] = models_glm_hmm[n_states][best_idx].transitions.params

In [27]:
# session-wise fitting with 5 fold cross-validation
models_session_state_fold, train_ll_session, test_ll_session = session_wise_fit_cv(choices_session_wise, inputs_session_wise, masks=masks_session_wise,
                                                                                    n_sessions=len(metadata.session_id.unique()), init_params=init_params, n_iters= 1000)

Fitting session 0...
Fitting 2 states...


LP: -336.3:   0%|          | 0/1000 [00:00<?, ?it/s]

Fitting 3 states...


LP: -376.3:   0%|          | 0/1000 [00:00<?, ?it/s]

Fitting 4 states...


...

LP: -362.0:   1%|▏         | 13/1000 [00:00<00:07, 124.29it/s]

Fitting 4 states...


LP: -407.0:   0%|          | 0/1000 [00:00<?, ?it/s]

Fitting 5 states...


Converged to LP: -383.7:  21%|██▏       | 214/1000 [00:01<00:05, 139.81it/s]


In [35]:
results = {
    'global':{
		'inputs': inputs_aggregate,
		'choices': choices_aggregate,
		'masks': masks_aggregate,
		'models': models_glm_hmm,
		'fit_lls': fit_lls_glm_hmm,
		'best_params': init_params
	},
	'session_wise':{
		'session_ids': metadata.session_id.unique(),
        'unnormalized_inputs': unnormalized_inputs,
		'inputs': inputs_session_wise,
		'choices': choices_session_wise,
		'masks': masks_session_wise,
		'reaction_time': reaction_time_session_wise,
		'models': models_session_state_fold,
		'train_ll': train_ll_session,
		'test_ll': test_ll_session
	}
}


with open(Path(processed_dir, f'glm_hmm_result.pkl'), 'wb') as f:
    pickle.dump(results, f)